# Imports

In [ ]:
import pandas as pd
from datetime import timedelta

In [ ]:
df=pd.read_csv('../data/raw_data/historical_flights.csv')

In [ ]:
df

## Fjerner kansellerte fly

In [ ]:
df = df[df["cancelled"] == 0].copy()

## Koverterer til datetime

In [ ]:
df['std'] = pd.to_datetime(df['std'])
df['sta'] = pd.to_datetime(df['sta'])
df['atd'] = pd.to_datetime(df['atd'])
df['ata'] = pd.to_datetime(df['ata'])

## Lager intervaller for kommunikasjon

In [ ]:
dep = df.dropna(subset=["atd"]).copy()
dep["start"] = dep["atd"] - pd.to_timedelta(15, "m")
dep["end"]   = dep["atd"] + pd.to_timedelta(8, "m")
dep["airport_group"] = dep["dep_airport_group"]

arr = df.dropna(subset=["ata"]).copy()
arr["start"] = arr["ata"] - pd.to_timedelta(16, "m")
arr["end"]   = arr["ata"] + pd.to_timedelta(5, "m")
arr["airport_group"] = arr["arr_airport_group"]

intervals = pd.concat([dep, arr], ignore_index=True)
intervals = intervals.dropna(subset=["airport_group"])

## Markerer overlapp

In [ ]:
def fast_overlap(group):
    events = []
    for idx, row in group.iterrows():
        events.append((row["start"], +1, idx))
        events.append((row["end"], -1, idx))
    events.sort()

    active = set()
    overlap_flags = {idx: 0 for idx in group.index}

    for time, change, idx in events:
        if change == +1:  # start
            if active:  # minst én er aktiv
                overlap_flags[idx] = 1
                for other in active:
                    overlap_flags[other] = 1
            active.add(idx)
        else:  # slutt
            active.remove(idx)

    group["target"] = group.index.map(overlap_flags)
    return group

In [ ]:
results = intervals.groupby("airport_group", group_keys=False).apply(fast_overlap)


In [ ]:
group_a=results[results["airport_group"] == 'A']